In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [25]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

In [26]:
dfs = [pd.read_csv('user_'+ user + '.csv') for user in ['a', 'b', 'c', 'd']]


In [27]:
for i in range(len(dfs)):
    dfs[i]['User'] = pd.Series(i, index=dfs[i].index)

In [28]:
data = pd.concat(dfs, axis=0).sample(frac=1.0, random_state=123).reset_index(drop=True)

In [29]:
data

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User
0,0.0,3572.252440,2065.056469,0.851824,2.047953,0.651871,2.522036,2.109733,3.347705,3564.779879,...,35.151586,3628.426885,2129.789645,5.353671,17.885132,7.672209,29.960618,43.216980,43.932669,0
1,1.0,3574.116024,2065.528155,1.932513,3.321636,1.138012,2.349805,2.256212,3.945981,3563.399422,...,40.800889,3680.341349,2144.200503,10.819521,36.995982,12.812193,24.146774,23.747501,49.072017,0
2,0.0,3554.487593,2056.215665,0.935015,3.793783,0.736168,2.680542,3.381325,4.678876,3568.839949,...,18.176841,3538.347368,2081.315814,5.486555,13.204753,0.664075,6.633072,1.434277,4.132446,2
3,0.0,3570.668125,2063.974908,1.875394,3.028541,0.910000,3.018672,1.163312,2.883009,3565.403408,...,55.547547,3604.601528,2122.493834,15.611283,16.452483,22.462175,46.703612,32.213578,70.892466,0
4,0.0,3559.747108,2057.401763,1.053691,1.530594,1.593121,2.789907,2.668865,4.326693,3573.651774,...,7.208052,3513.244789,2030.461207,1.455450,3.030659,0.482971,3.200647,0.895170,2.099638,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,2.0,3570.748191,2063.201099,0.727594,1.404708,0.995674,2.569811,0.991322,1.987807,3562.594707,...,1.910036,3567.687654,2062.791757,1.748982,2.097780,1.242056,1.682180,1.277379,1.997654,3
11516,1.0,3566.600695,2062.436502,1.491797,3.013466,1.338308,2.769207,4.268401,3.877338,3576.187723,...,5.217772,3548.277991,2055.537892,1.787661,3.366967,0.810903,1.739812,2.202195,3.294145,2
11517,0.0,3574.186933,2064.477869,0.710708,1.534898,0.749206,2.038258,2.208018,2.157076,3566.398415,...,2.200203,3572.290285,2063.776063,1.268569,2.547969,0.887808,1.651011,1.533523,1.957866,1
11518,0.0,3574.343116,2065.996679,0.606401,2.835015,0.617079,1.209722,3.357912,3.098423,3566.521533,...,15.413625,3608.394990,2091.509508,1.479338,3.824176,3.281520,4.238889,33.441194,19.158094,0


In [30]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [31]:
def preprocess_inputs(df, target='Class'):
    df = df.copy()
    
    # One-hot encode whichever target column is not being used
    targets = ['Class', 'User']
    targets.remove(target)
    df = onehot_encode(df, column=targets[0])
    
    # Split df into X and y
    y = df[target].copy()
    X = df.drop(target, axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=50)
    
    # Scale X with a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    
    return X_train, X_test, y_train, y_test

In [32]:
def build_model(num_classes=4):
    
    inputs = tf.keras.Input(shape=(None, 116))
    x = tf.keras.layers.Reshape((-1, 116))(inputs)
    x = tf.keras.layers.LSTM(256, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(128, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(64)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [33]:
X_train, X_test, y_train, y_test = preprocess_inputs(data, target='Class')

In [37]:
# class_model = build_model(num_classes=4)
class_model = build_model(num_classes=4)
   
class_model.summary()
#     plot_model(class_model, to_file='model_diagram.png', show_shapes=True, show_layer_names=True)

class_history = class_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=5,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None, 116)]       0         
                                                                 
 reshape_4 (Reshape)         (None, None, 116)         0         
                                                                 
 lstm_12 (LSTM)              (None, None, 256)         381952    
                                                                 
 lstm_13 (LSTM)              (None, None, 128)         197120    
                                                                 
 lstm_14 (LSTM)              (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 4)                 260       
                                                                 
Total params: 628,740
Trainable params: 628,740
Non-trainab

In [38]:
class_acc = class_model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy (Class Model): {:.2f}%".format(class_acc * 100))

Test Accuracy (Class Model): 67.06%
